In [ ]:
from dotenv import load_dotenv
from pprint import pprint 
from langchain_community.utilities import SQLDatabase

load_dotenv()


db = SQLDatabase.from_uri("sqlite:///db/db/horror.db")

# 以游标身份查询
result = db.run("SELECT content FROM Artist LIMIT 12;", fetch="cursor")
print(type(result))
pprint(list(result.mappings()))


In [6]:
import sqlalchemy as sa
from pprint import pprint
from langchain_community.utilities import SQLDatabase


db = SQLDatabase.from_uri("sqlite:///db/db/horror.db")
metadata = sa.MetaData()
lingyizhi = sa.Table(
    "lingyizhi",
    metadata,
    sa.Column("id", sa.INTEGER, primary_key=True),
    sa.Column("title", sa.TEXT),
    sa.Column("content", sa.TEXT),
)

keywords = ["公交", "地铁", "出租车", "卡车", "火车", "货车", "动车", "高铁", "飞机", "灵车"]

conditions = [lingyizhi.c.title.like(f'%{keyword}%') for keyword in keywords]
query = sa.select(lingyizhi).where(sa.or_(*conditions))

result = db.run(query, fetch="cursor")
pprint(list(result.mappings()))



[{'id': 207, 'title': '送灵车内车的鬼影（续）', 'content': '送灵车内的鬼影\r\n                \r\n                    懒鬼先生\r\n                    \r\n                        \r\n                            20年12月1日\r\n                        \r\n                        \r\n                            喜欢：3\r\n                        \r\n                        \r\n                            浏览：2.6k\r\n                        \r\n                    \r\n                \r\n            \n第二天，一早拉长带着我去了一趟集市购买一些做法事的东西。\n路上我和拉长讨论着，今天中午给她做了法事后她是不是就能进屋了 还能和他们那些已故的长辈们共用香炉？\n拉长边挑东西边笑呵呵的说 小李啊 你可能看多鬼故事或者电影了，有些事情啊往往看似简单却不以为然啊 要想能共用一个香炉还要等入土后 做最后的事给他们那些祖宗啥的念叨念叨啊哈的 才行呢。\n那今儿的法事是做什么，拉长继续说道 今儿做的法事是 让她本人知道自己已故 人走阳关道 鬼走阴关路呀 不要继续存在人家或者粘着亲人不放 这样只会让她亲人不好反而会害了他们。。\n等快到了中午的时候 拉长开着他的小绵羊电动车慢悠悠带着我赶到了客户家里 吃了午饭就开始摆坛做法事，拉长穿上了道袍手抓着那个啥 佛尘\n在法坛面前走起了七星步伐 抓着 佛尘 用底部对着法坛画着 嘴里念着道语 原地一踏 喊着 亲人 跪 什么三拜九叩的\xa0 站在了旁边念着\xa0 白色纸写的东西 然后又是三拜九叩 走到死者门面前还要下跪叩首。。。最后拉长喊着一句放炮。。😂😂\xa0 我一下反应不过来还以为是干嘛呢 结果要我去放鞭炮。。。\n当法事做完后，拉长要我把临时灵位摆在死者脚下 上柱香 再回来 我起身后眼角 突然看到死者

In [5]:
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_fireworks import ChatFireworks
import re
from dotenv import load_dotenv
import os

load_dotenv()
llm = ChatFireworks(model="accounts/fireworks/models/llama-v3-70b-instruct",temperature=0.3)
prompt = ChatPromptTemplate.from_messages([SystemMessagePromptTemplate.from_template("你是一名灵异故事创作者。"),
                                           HumanMessagePromptTemplate.from_template(
"""
请写一篇关于{input}的5000字灵异故事。
输出格式：

- 标题（汉语）:
- 汉语关键词（汉语 1-3个）:
- 英语关键词（汉语 1-3个）:
- 内容（汉语）:

""")])
# 创建链
chain = prompt | llm | StrOutputParser()
# 调用链
input = "两个宋家庄地铁站"
content = chain.invoke({"input": input})
content = re.sub('\n+','\n',content)
print(content)

HTTPStatusError: Client error '412 Precondition Failed' for url 'https://api.fireworks.ai/inference/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/412